# Credit Risk Analysis

## Import packages

1. `sys`: System-specific parameters and functions.
2. `reload` (from `imp`): Reload previously imported modules.
3. `matplotlib.pyplot`: Data visualization.
4. `numpy`: Numerical computing.
5. `pandas`: Data manipulation and analysis.
6. `seaborn`: Statistical data visualization.
7. `SimpleImputer` (from `sklearn.impute`): Handling missing data.
8. `LogisticRegression` (from `sklearn.linear_model`): Logistic regression for classification.

In [12]:
import sys

sys.path.append("..")

from imp import reload

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

from helper_functions import config, data_utils, evaluation, plot, preprocessing

In [13]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

## Load normalized data set


In this notebook, we are going to encode a previously normalized `dataset` followed by the creation of the `ML` model.

In [14]:
app_normalized = data_utils.get_normalized_model()
app_normalized['TARGET_LABEL_BAD=1'] = app_normalized.pop('TARGET_LABEL_BAD=1')

In [15]:
app_normalized.head()

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,RESIDENCIAL_STATE,FLAG_RESIDENCIAL_PHONE,MONTHS_IN_RESIDENCE,FLAG_EMAIL,COMPANY,...,PRODUCT,AGE,HAS_DEPENDANTS,HAS_RESIDENCE,MONTHLY_INCOMES_TOT,HAS_CARDS,HAS_BANKING_ACCOUNTS,HAS_PERSONAL_ASSETS,HAS_CARS,TARGET_LABEL_BAD=1
0,1 - 14,Web,F,other,1,RN,Y,+ 1 year,1,N,...,1,26 - 35,True,True,[650 - 1320],True,False,False,False,1
1,15 - 30,Carga,F,married,0,RJ,Y,0 - 6 months,1,Y,...,1,26 - 35,False,True,[650 - 1320],False,False,False,False,1
2,1 - 14,Web,F,married,0,RN,Y,+ 1 year,1,N,...,1,26 - 35,False,True,[0 - 650],False,False,False,False,0
3,15 - 30,Web,F,married,0,PE,N,+ 1 year,1,N,...,1,> 60,False,False,[0 - 650],False,False,False,False,0
4,1 - 14,Web,M,married,0,RJ,Y,6 months - 1 year,1,N,...,1,46 - 60,False,True,[650 - 1320],False,False,False,False,1
